In [37]:
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.llms import HuggingFaceHub
from huggingface_hub import InferenceClient
from dotenv import load_dotenv

load_dotenv();

In [38]:
config = {'max_new_tokens': 256, 'temperature': 0.1}

In [39]:
# Calling llama model

# llm = CTransformers(model="D:\Code Workspace\DL Model\llama-2-7b-chat.ggmlv3.q8_0.bin",
#                     model_type = 'llama',
#                     config = config)

llm = HuggingFaceHub(
    repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1",
    model_kwargs = config
)

# llm = InferenceClient(
#     "mistralai/Mixtral-8x7B-Instruct-v0.1"
# )


d:\Code Workspace\Personal Project\Personal Project - LangChain Project\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [40]:
llm.predict("testing")

'\n\n# Testing\n\nTesting is an important part of the development process. It ensures that the software is working as intended and helps to identify any bugs or issues that need to be fixed.\n\nThere are several types of testing that can be performed, including:\n\n* **Unit testing**: This involves testing individual units or components of the software to ensure that they are working correctly. Unit tests are typically written by developers and are automated, meaning that they can be run automatically as part of the build process.\n* **Integration testing**: This involves testing how different units or components of the software work together. Integration tests are also typically automated and are run as part of the build process.\n* **System testing**: This involves testing the software as a whole to ensure that it is functioning correctly. System tests are typically performed by a separate team of testers and may involve manual testing as well as automated tests.\n* **Acceptance test

In [32]:
## Create LLM Chaining
questions_template = "Generate a {selected_topic_level} math quiz on the topic of {selected_topic}. Include {num_quizzes} questions"
questions_prompt = PromptTemplate(input_variables=["selected_topic_level", "selected_topic", "num_quizzes"],
                                    template=questions_template)
questions_chain = LLMChain(llm= llm,
                            prompt = questions_prompt,
                            output_key = "questions")

In [33]:
answer_template = "From this Question:\n {questions}\n\n gave me answer to each one of them"
answer_prompt = PromptTemplate(input_variables = ["questions"],
                                template = answer_template)
answer_chain = LLMChain(llm = llm,
                        prompt = answer_prompt,
                        output_key = "answer")

In [34]:
## Create Sequential Chaining
seq_chain = SequentialChain(chains = [questions_chain, answer_chain],
                            input_variables = ['selected_topic_level', 'selected_topic', 'num_quizzes'],
                            # Here we return multiple variables
                            output_variables = ['questions', 'answer'])

In [35]:
response = seq_chain({'selected_topic_level': 'Elementary School Level', 
                            'selected_topic': 'Fraction', 
                            'num_quizzes' : 3})

In [36]:
response

{'selected_topic_level': 'Elementary School Level',
 'selected_topic': 'Fraction',
 'num_quizzes': 3,
 'questions': ' on the topic of Fraction.\n\n1. What is a fraction?\na. A whole number\nb. A part of a whole number\nc. A number greater than 1\n\nAnswer: b. A part of a whole number\n\n2. What is the fraction of the shaded part in the figure below?\n\na. 1/4\nb. 1/2\nc. 3/4\n\nAnswer: b. 1/2\n\n3. If John ate 3/4 of a pizza, how much pizza did he eat?\na. 1/4\nb. 1/2\nc. 3/4\n\nAnswer: c. 3/4',
 'answer': '.\n\n1. What is a mixed number?\na. A whole number\nb. A part of a whole number\nc. A number that has both a whole number and a fraction\n\nAnswer: c. A number that has both a whole number and a fraction\n\n2. What is the mixed number of the figure below?\n\na. 1 1/4\nb. 1 1/2\nc. 1 3/4\n\nAnswer: b. 1 1/2\n\n3. If John ate 1 1/2 pizzas, how much pizza did he eat in total?\na. 1/2\nb. 1 1/2\nc. 3\n\nAnswer: c. 3\n\n4. What is an improper fraction?\na. A fraction that is greater than